## First, let's install and import all required libraries

In [1]:
#!pip install -r requirements.txt

In [2]:
import collections                        #This library adds some extras to the starndard python data structures
import folium                             #Great library for plotting data on maps
import json                               #This library allows us to load and hyndle json files with python
import os                                 #This module is usefull for handling paths and directories
import webbrowser                         #This library allows to handle the webbrowser from python
import pandas as pd
from folium.plugins import HeatMap
from progressbar import ProgressBar, Bar, ETA, Percentage
from datetime import datetime
#from utils import timestampInRange

## Now, let's load and analyze our location history json file 

In [3]:
data = json.load(open('Location_History_Juan.json'))

In [4]:
data

{'locations': [{'timestampMs': '1354262153219',
   'latitudeE7': 524535915,
   'longitudeE7': 132941230,
   'accuracy': 32},
  {'timestampMs': '1354271661093',
   'latitudeE7': 524533301,
   'longitudeE7': 132934986,
   'accuracy': 32},
  {'timestampMs': '1354287407617',
   'latitudeE7': 524533301,
   'longitudeE7': 132934986,
   'accuracy': 32},
  {'timestampMs': '1354291299371',
   'latitudeE7': 524534106,
   'longitudeE7': 132921642,
   'accuracy': 62},
  {'timestampMs': '1354291359891',
   'latitudeE7': 524537807,
   'longitudeE7': 132911611,
   'accuracy': 52,
   'activity': [{'timestampMs': '1354291328356',
     'activity': [{'type': 'STILL', 'confidence': 100}]}]},
  {'timestampMs': '1354291420670',
   'latitudeE7': 524537807,
   'longitudeE7': 132911611,
   'accuracy': 52},
  {'timestampMs': '1354291600666',
   'latitudeE7': 524535330,
   'longitudeE7': 132926220,
   'accuracy': 55,
   'activity': [{'timestampMs': '1354291525216',
     'activity': [{'type': 'STILL', 'confidence

In [5]:
type(data)

dict

In [6]:
data["locations"]

[{'timestampMs': '1354262153219',
  'latitudeE7': 524535915,
  'longitudeE7': 132941230,
  'accuracy': 32},
 {'timestampMs': '1354271661093',
  'latitudeE7': 524533301,
  'longitudeE7': 132934986,
  'accuracy': 32},
 {'timestampMs': '1354287407617',
  'latitudeE7': 524533301,
  'longitudeE7': 132934986,
  'accuracy': 32},
 {'timestampMs': '1354291299371',
  'latitudeE7': 524534106,
  'longitudeE7': 132921642,
  'accuracy': 62},
 {'timestampMs': '1354291359891',
  'latitudeE7': 524537807,
  'longitudeE7': 132911611,
  'accuracy': 52,
  'activity': [{'timestampMs': '1354291328356',
    'activity': [{'type': 'STILL', 'confidence': 100}]}]},
 {'timestampMs': '1354291420670',
  'latitudeE7': 524537807,
  'longitudeE7': 132911611,
  'accuracy': 52},
 {'timestampMs': '1354291600666',
  'latitudeE7': 524535330,
  'longitudeE7': 132926220,
  'accuracy': 55,
  'activity': [{'timestampMs': '1354291525216',
    'activity': [{'type': 'STILL', 'confidence': 100}]}]},
 {'timestampMs': '1354291661872'

In [7]:
df = pd.DataFrame(data["locations"])

In [8]:
df = df.drop(columns=['activity'])
df

,timestampMs,latitudeE7,longitudeE7,accuracy
0,1354262153219,524535915,132941230,32
1,1354271661093,524533301,132934986,32
2,1354287407617,524533301,132934986,32
3,1354291299371,524534106,132921642,62
4,1354291359891,524537807,132911611,52
...,...,...,...,...
205292,1500397386151,481479476,115870654,10
205293,1500404117256,481168069,115342124,10
205294,1500407503014,481146421,115321200,41
205295,1500407508999,481144178,115320414,10


## Now we can extract and tranform the data we need to make our map 

In [9]:
coordinates = collections.defaultdict(int)

In [10]:
coordinates

defaultdict(int, {})

In [11]:
max_coordinates = (0, 0)

In [12]:
max_magnitude = 0

In [13]:
"""Here you can introduce a range of dates for the map. for example date_range=('2000-01-02', '2019-12-31')"""


def timestampInRange(timestamp, date_range):
    """Returns if the timestamp is in the date range.

    Arguments:
        timestamp {str} -- A timestamp (in ms).
        date_range {tuple} -- A tuple of strings representing the date range.
        (min_date, max_date) (Date format: yyyy-mm-dd)
    """
    if date_range == (None, None):
        return True
    date_str = datetime.fromtimestamp(
        int(timestamp) / 1000).strftime("%Y-%m-%d")
    

    return dateInRange(date_str, date_range)

def dateInRange(date, date_range):
    """Returns if the date is in the date range.

    Arguments:
        date {str} -- A date (Format: yyyy-mm-dd).
        date_range {tuple} -- A tuple of strings representing the date range.
        (min_date, max_date) (Date format: yyyy-mm-dd)
    """
    if date_range == (None, None):
        return True
    if date_range[0] == None:
        min_date = None
    else:
        min_date = datetime.strptime(date_range[0], "%Y-%m-%d")
    if date_range[1] == None:
        max_date = None
    else:
        max_date = datetime.strptime(date_range[1], "%Y-%m-%d")
    date = datetime.strptime(date, "%Y-%m-%d")
    return (min_date is None or min_date <= date) and \
        (max_date is None or max_date >= date)



date_range = ('2016-01-02', '2020-12-31')

In [14]:
"""Here you transfor the coordinates given by google in actual longitude and latitude coordinates and
show the progress using the progress bar widget"""

w = [Bar(), Percentage(), " ", ETA()]
with ProgressBar(max_value=len(data["locations"]), widgets=w) as pb:
    for i, loc in enumerate(data["locations"]):
        # print(i)
        # print(loc)
        if "latitudeE7" not in loc or "longitudeE7" not in loc:
            continue
        coords = (round(loc["latitudeE7"] / 1e7, 6),
                    round(loc["longitudeE7"] / 1e7, 6))
        # print(coords)

        
        """Here you calculate the magnitude for all coordinates within the date range"""
        
        if timestampInRange(loc["timestampMs"], date_range): 
            #print(loc["timestampMs"])
            coordinates[coords] += 1 #these are the magnitude values we will need for the coordinates dictionary
            #print(coordinates[coords])
            
            if coordinates[coords] > max_magnitude:
                max_coordinates = coords
                max_magnitude = coordinates[coords]
                #print(max_coordinates)
                #print(max_magnitude)
        
        """This command updates the progress bar"""
        pb.update(i)

|##########################################################|100% Time:  0:00:19


In [15]:
coordinates

defaultdict(int,
            {(50.943394, 6.963126): 5,
             (50.943419, 6.963124): 1,
             (50.943435, 6.963092): 1,
             (50.943423, 6.963082): 1,
             (50.943462, 6.963054): 4,
             (50.943453, 6.963002): 1,
             (50.942087, 6.962696): 1,
             (50.938366, 6.962682): 1,
             (50.936611, 6.962417): 1,
             (50.935775, 6.959996): 1,
             (50.935401, 6.949204): 1,
             (50.936481, 6.939018): 1,
             (50.936707, 6.931622): 1,
             (50.936687, 6.919985): 1,
             (50.936797, 6.91507): 1,
             (50.937435, 6.894011): 1,
             (50.937669, 6.86845): 1,
             (50.938089, 6.854992): 1,
             (50.938322, 6.857228): 1,
             (50.938325, 6.85723): 1,
             (50.938228, 6.857139): 1,
             (50.938222, 6.857213): 2,
             (50.938329, 6.857227): 1,
             (50.938009, 6.855545): 1,
             (50.937772, 6.855823): 1,
           

In [16]:
tilesoptions = ["openstreetmap", "StamenTerrain", "stamentoner",
                "stamenwatercolor", "cartodbpositron", "cartodbdark_matter"]

tiles = tilesoptions[5]
zoom_start = 10
radius = 7
blur = 4
min_opacity = 0.2
max_zoom = 10


In [17]:
map_data = [(coords[0], coords[1], magnitude)
            for coords, magnitude in coordinates.items()] #coords are keys in the dict. magnitudes are the values

In [18]:
map_data

[(50.943394, 6.963126, 5),
 (50.943419, 6.963124, 1),
 (50.943435, 6.963092, 1),
 (50.943423, 6.963082, 1),
 (50.943462, 6.963054, 4),
 (50.943453, 6.963002, 1),
 (50.942087, 6.962696, 1),
 (50.938366, 6.962682, 1),
 (50.936611, 6.962417, 1),
 (50.935775, 6.959996, 1),
 (50.935401, 6.949204, 1),
 (50.936481, 6.939018, 1),
 (50.936707, 6.931622, 1),
 (50.936687, 6.919985, 1),
 (50.936797, 6.91507, 1),
 (50.937435, 6.894011, 1),
 (50.937669, 6.86845, 1),
 (50.938089, 6.854992, 1),
 (50.938322, 6.857228, 1),
 (50.938325, 6.85723, 1),
 (50.938228, 6.857139, 1),
 (50.938222, 6.857213, 2),
 (50.938329, 6.857227, 1),
 (50.938009, 6.855545, 1),
 (50.937772, 6.855823, 1),
 (50.938037, 6.855143, 1),
 (50.93785, 6.855295, 1),
 (50.937996, 6.855457, 1),
 (50.937567, 6.871018, 1),
 (50.937131, 6.90516, 1),
 (50.936686, 6.917008, 1),
 (50.935816, 6.932597, 1),
 (50.935901, 6.933069, 1),
 (50.938214, 6.934464, 1),
 (50.93842, 6.934424, 1),
 (50.938354, 6.934277, 1),
 (50.938421, 6.934432, 1),
 (50.93

In [19]:
# Generate background map
m = folium.Map(location=max_coordinates,
                zoom_start=zoom_start,
                tiles=tiles)

# Generate heat map
heatmap = HeatMap(map_data,
                    max_val=max_magnitude,
                    min_opacity=min_opacity,
                    radius=radius,
                    blur=blur,
                    max_zoom=max_zoom)

m.add_child(heatmap)

## Finally, we can save our map as an html file and launch it in the browser 

In [20]:
output_file = tiles + 'heatmapJuan.html'

In [21]:
m.save(output_file)

In [22]:
webbrowser.open("file://" + os.path.realpath(output_file))

True